In [ ]:
import sys
sys.path.append('../htm_rl/htm_rl/')

from agent.agent import Agent, AgentRunner
from agent.memory import Memory, TemporalMemory
from agent.planner import Planner
from common.sa_sdr_encoder import SaSdrEncoder, format_sa_superposition
from common.base_sa import SaRelatedComposition, Sa, SaSuperposition
from common.int_sdr_encoder import IntSdrEncoder, IntRangeEncoder
from common.int_sdr_encoder import SequenceSdrEncoder
from envs.gridworld_pomdp import GridWorld


import numpy as np
import matplotlib.pyplot as plt

Базовый тест памяти с использованием новой кодироки состояний.

In [ ]:
world_description = [[2,0,0],
                     [1,1,0],
                     [0,0,0]]

In [ ]:
gw = GridWorld(world_description, (3, 3), agent_initial_position={'row': 2, 'column': 0})

In [ ]:
gw.world_size

In [ ]:
gw.render()

In [ ]:
max_steps = 12

state_encoder = SequenceSdrEncoder('state',
                                   encoders=[IntSdrEncoder('distance',
                                                                   gw.world_size[0],
                                                                   5,
                                                                   4),
                                             IntSdrEncoder('surface', 3, 5, 4),
                                             IntRangeEncoder('row', (-(gw.world_size[0]-1),
                                                                     gw.world_size[1]-1), 5, 4),
                                             IntRangeEncoder('column', (-(gw.world_size[0]-1),
                                                                     gw.world_size[1]-1), 5, 4),
                                             IntSdrEncoder('direction', 4, 5, 4)],
                                   size=5)

In [ ]:
print(state_encoder.value_bits)
state_encoder.total_bits

In [ ]:
action_encoder = IntSdrEncoder('action', gw.n_actions,
                              value_bits=6, activation_threshold=4)

In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
sa_encoder.total_bits, sa_encoder.value_bits, sa_encoder.activation_threshold

In [ ]:
action_encoder.activation_threshold

In [ ]:
sa_encoder.value_bits ** 2

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=8,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=sa_encoder.activation_threshold,
                    initial_permanence=0.5,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits*2,
                    permanenceIncrement=0.1,
                    permanenceDecrement=0.02,
                    predictedSegmentDecrement=0.001
                    )

In [ ]:
tm.activation_threshold, sa_encoder.value_bits

In [ ]:
tm.getMaxSegmentsPerCell(), tm.getMaxNewSynapseCount(), tm.getMaxSynapsesPerSegment()


In [ ]:
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
actions_basic = [2, 2, 1, 2, 2, 1, 2, 2]

In [ ]:
def run_test(actions, temporal_memory, environment, verbosity=1, n_cycles=3, with_reset=False):
    for i in range(n_cycles):
        if with_reset:
            temporal_memory.reset()
        if verbosity>1:
            print()
            print(f'*** cycle {i+1} ***')
            print()
        state, reward, done = environment.reset(), 0, False
        for action in actions:
            if verbosity > 1:
                environment.render()
                print(f'Action {action} State: {state}')
            temporal_memory.train(Sa(state, action), verbosity)
            state, _, _, info = environment.step(action)

In [ ]:
run_test(actions_basic, memory, gw, n_cycles=3, verbosity=3, with_reset=True)

Пробуем воспроизвести ситуацию, когда память не может предсказать последовательность.

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=8,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=action_encoder.activation_threshold,
                    initial_permanence=0.5,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits,
                    permanenceIncrement=0.1,
                    permanenceDecrement=0,
                    predictedSegmentDecrement=0
                    )
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
run_test(actions_basic, memory, gw, verbosity=3)

In [ ]:
action_encoder.activation_threshold

Пытаемся объяснить, почему так происходит.

За что отвечает learning_threshold?

learning_threshold = minThreshold

minThreshold это минимальное количество активных синапсов сегмента при котором будет происходить их обучение.

Посмотрим, как изменится ситуация, если начальное знанчение permanence сделать заведомо больше чем нужно
для соединения

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=8,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=action_encoder.activation_threshold,
                    initial_permanence=1,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits,
                    permanenceIncrement=0.1,
                    permanenceDecrement=0,
                    predictedSegmentDecrement=0
                    )
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
run_test(actions_basic, memory, gw, verbosity=3)


Ситуация не изменилась. Возможно, есть ещё параметры, которые могут влиять на permanence.
В противном случае непонятно, почему последовательность не запоминается.

Установим learning_threshold = 0

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=8,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=0,
                    initial_permanence=0.5,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits,
                    permanenceIncrement=0.0,
                    permanenceDecrement=0.0,
                    predictedSegmentDecrement=0.0
                    )
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
run_test(actions_basic, memory, gw, verbosity=3, n_cycles=3)

In [ ]:
tm.printParameters()


В крайнем случае, когда learning_threshold=0, выигрывают одни и те же клетки, которые уже имеют сегменты. Это понятно
из алгоритма. Однако пока непонятно, почему алгоритм каждый раз создаёт новые синапсы и дропает предыдущие. По идее,
он должен просто реинфорсить старые синапсы и наращивать новые, если слоты ещё не заполнены.

Из кода на github понятно, что в случае, если не хватает слотов,
то удаляют синапсы с наименьшим permanence, чтобы освобоить место для
новых синапсов. Но всё же не ясно, как так получается, что nGrowDesired>0 (`#228, Temporal_Memory.cpp`).

`nGrowDesired = maxNewSynapseCount_ - numActivePotentialSynapsesForSegment_[*bestMatchingSegment]`

Итак, вероятно, где-то баг или я не так что-то понял, но суть проблемы в том, что когда у нас learning_threshold=0 и
sdr сильно перекрываются, т.е. меняются мало, то на каджом шаге часто активируются одни и те же колонки,
среди которых выбираются одни и те же клетки, т.к. они все имеют достаточно потенциально активных сегментов
в силу условия learning_threshold=0. Баг в том, что
по идее, эти выбранные сегменты просто должны реинфорсить свои синапсы или наказывать их, и т.к. все слоты уже заняты,
число новых синапсов должно быть равным нулю, но на деле, получается, что выращиваются новые синапсы, которые переподключаются
к новым клеткам, а старые синапсы удаляются. Такие клетки никогда не смогут выучить свой контекст. Из кода следует, что если число
потенциальных синапсов с активной предсинаптической клеткой меньше максимально добавляемого числа синапсов,
то будут выращены новые синапсы,
причём, если им не хватает слотов на сегменте, то будут удалены старые, наименее активные синапсы, с наименьшим permanence.
Что здесь и происходит. Т.к. не все предсинаптические клетки сегмента активны в силу того, что sdr всё-таки разные, то вот эта
неактивная часть синапсов будет перезаписана новыми синапсами. То есть, как раз та часть перезапишется, которая